<a href="https://colab.research.google.com/github/Kamalesh-Swargam/-dockingbay/blob/main/comfyui_colab_Flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the [ComfyUI](https://github.com/comfyanonymous/ComfyUI) repo and install the requirements.

In [1]:
!git clone -q https://github.com/comfyanonymous/ComfyUI
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu130
%pip install -q -r ./ComfyUI/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu130
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.6/680.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3

In [2]:
# Some Helpful Custom Nodes
!git clone -q https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/comfyui-manager

!git clone -q https://github.com/ClownsharkBatwing/RES4LYF.git /content/ComfyUI/custom_nodes/RES4LYF
!git clone -q https://github.com/rgthree/rgthree-comfy.git /content/ComfyUI/custom_nodes/rgthree-comfy
!git clone -q https://github.com/yolain/ComfyUI-Easy-Use.git /content/ComfyUI/custom_nodes/ComfyUI-Easy-Use

Download the vae and dual text encoder needed for FLUX - (Compulsory)

In [3]:
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors -P /content/ComfyUI/models/vae/

# 8-step distilled lora for fast inference on dev models
# !wget -q --show-progress https://huggingface.co/alimama-creative/FLUX.1-Turbo-Alpha/resolve/main/diffusion_pytorch_model.safetensors -P /content/ComfyUI/models/loras

clip_l.safetensors  100%[===================>] 234.74M  42.7MB/s    in 6.6s    
t5xxl_fp8_e4m3fn.sa 100%[===================>]   4.56G  63.5MB/s    in 88s     
ae.safetensors      100%[===================>] 319.77M  72.4MB/s    in 5.1s    


In [4]:
%pip install --upgrade gguf
!rm -rf ./custom_nodes/ComfyUI-GGUF
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.8 MB/s eta 0:00:00
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-GGUF'...
remote: Enumerating objects: 814, done.
remote: Counting objects: 100% (508/508), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 814 (delta 458), reused 312 (delta 312), pack-reused 306 (from 2)
Receiving objects: 100% (814/814), 189.66 KiB | 3.65 MiB/s, done.
Resolving deltas: 100% (543/543), done.


In [5]:
!mv /content/ComfyUI/models/diffusion_models/flux1-dev-Q4_0.gguf /content/ComfyUI/models/unet/flux1-dev-Q4_0.gguf

mv: cannot stat '/content/ComfyUI/models/diffusion_models/flux1-dev-Q4_0.gguf': No such file or directory


### Run ComfyUI with ngrok

In [ ]:
!pip install -q pyngrok

from pyngrok import ngrok
import subprocess, socket, time
from google.colab import userdata

NGROK_TOKEN = userdata.get("NGROK_TOKEN")

# Set ngrok token
!ngrok config add-authtoken $NGROK_TOKEN

# Start ComfyUI in background
subprocess.Popen(["python", "/content/ComfyUI/main.py", "--dont-print-server"])

# Wait until port is open
port = 8188
while True:
    try:
        sock = socket.create_connection(("127.0.0.1", port), timeout=2)
        sock.close()
        print("✅ ComfyUI server is running on port", port)
        break
    except OSError:
        print("⏳ Waiting for ComfyUI to start...")
        time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(8188, bind_tls=True)
print("🌐 Public URL:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for ComfyUI to start...
⏳ Waiting for